In [1]:
from konlpy.tag import Mecab
mecab = Mecab()

print(mecab.pos('대한민국 만세 만셈 으어 나는 나넌 너의 친구야.'))

[('대한민국', 'NNP'), ('만세', 'NNG'), ('만', 'JX'), ('셈', 'NNG'), ('으어', 'IC'), ('나', 'NP'), ('는', 'JX'), ('나', 'NP'), ('넌', 'NP+JX'), ('너', 'NP'), ('의', 'JKG'), ('친구', 'NNG'), ('야', 'VCP+EF'), ('.', 'SF')]


In [3]:
print(f"pos 품사 추출 : {mecab.pos('중간고사 점수 내가 반에서 제일 잘 받음')}")
print(f"morphs 형태소 추출 : {mecab.morphs('중간고사 점수 내가 반에서 제일 잘 받음')}")
print(f"nouns 명사 추출 : {mecab.nouns('중간고사 점수 내가 반에서 제일 잘 받음')}")

pos 품사 추출 : [('중간고사', 'NNG'), ('점수', 'NNG'), ('내', 'NP'), ('가', 'JKS'), ('반', 'NNG'), ('에서', 'JKB'), ('제일', 'MAG'), ('잘', 'MAG'), ('받', 'VV'), ('음', 'ETN')]
morphs 형태소 추출 : ['중간고사', '점수', '내', '가', '반', '에서', '제일', '잘', '받', '음']
nouns 명사 추출 : ['중간고사', '점수', '내', '반']


In [ ]:
mecab.morphs()

In [14]:
import pandas as pd
df = pd.read_csv('/data/ephemeral/hsk/level1-semantictextsimilarity-nlp-01/data/train.csv')

In [15]:
def morphs_to_sentence(sentence):
    morphs = mecab.morphs(sentence)
    return ' '.join(morphs)

In [8]:
df['sentence_1'][0]

'스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~'

In [9]:
morphs_to_sentence(df['sentence_1'][0])

'스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하고 는 차원 이 다르 네요 ~'

In [16]:
# sentence_1과 sentence_2 칼럼에 대해 형태소 추출 및 변환
df['sentence_1'] = df['sentence_1'].apply(morphs_to_sentence)
df['sentence_2'] = df['sentence_2'].apply(morphs_to_sentence)



# 변환된 데이터를 새로운 CSV 파일로 저장
df.to_csv('train_morphs.csv', index=False)

# 결과 확인
df.head()

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴 도 있 고 반전 도 있 고 여느 한국 영화 쓰레기 들 하고 는 차원 이 다르 ...,"반전 도 있 고 , 사랑 도 있 고 재미 도 있 네요 .",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제 가 접근 권한 이 없 다고 뜹니다 ;;,"오 , 액세스 권한 이 없 다고 합니다 .",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택 청약 조건 변경 해 주 세요 .,주택 청약 무주택 기준 변경 해 주 세요 .,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사 후 처음 대면 으로 만나 반가웠 습니다 .,화상 으로 만 보 다가 리얼 로 만나 니 정말 반가웠 습니다 .,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯 뿌듯 하 네요 ! !,꼬옥 실제로 한 번 뵈 어요 뿌 뿌 뿌 ~!~!,0.0,0.0
